# Investigation on k-Nearest Neighbors Model

***By: Rabindra Nepal***


In this notebook, we explore the k-NN model based on different distance metrics: **Minkowski and Mahalanobis** distances. Also, we will look into the average performances of the model with two different distances. 

In [4]:
# Dependencies

import math
import time
import numpy as np
from numpy import linalg
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('_classic_test')
%matplotlib inline

# searborn: plotting
import seaborn as sns
from scipy import stats

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.preprocessing import scale

In [5]:
# warnings suppression
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

## k-NN Classification 

Here we will use the randomly created training and testing dataset. A data_generator function written below creates the required dataset - which can be controlled for repeated dataset using the random seed as a parameter. 

In [6]:
# returns X, y arrays 
def data_generator(seed=19):
    
    np.random.seed(seed)
    
    # Mean and covariance for Class 0
    mean0 = [0, 0, 0]
    cov0 = [[2550, 2000, 1500], [2000, 1500, 1200], [1500, 1200, 1900]]  


    # Number of datapoints for class 0
    m0 = 100


    # Generate class 0 data points from a multivariate (3D) Gaussian distribution
    #    Here x0_1, x0_2 and x0_3 are 3 dimensions for each data (feature) point

    x0_1, x0_2, x0_3 = np.random.multivariate_normal(mean0, cov0, m0).T


    # Concatenate the 3 dimensions of each feature to create the data matrix for class 0 
    X0 = np.concatenate((x0_1.reshape(-1, 1), x0_2.reshape(-1, 1), x0_3.reshape(-1, 1)), axis=1)

    # Create the target vector for class 0 (target is coded with zero)
    X0_target = np.zeros((m0,), dtype=np.int).reshape(-1, 1)



    # Mean and covariance for Class 1
    mean1 = [3, 3, 3]
    cov1 = [[2550, 2000, 1500], [2000, 1500, 1200], [1500, 1200, 1900]] 

    # Number of datapoints for class 1
    m1 = 100


    # Generate class 1 data points from a multivariate (3D) Gaussian distribution
    #    Here x1_1, x1_2 and x1_3 are 2 dimensions for each data (feature) point
    x1_1, x1_2, x1_3 = np.random.multivariate_normal(mean1, cov1, m1).T

    # Concatenate the 3 dimensions of each feature to create the data matrix for class 1
    X1 = np.concatenate((x1_1.reshape(-1, 1), x1_2.reshape(-1, 1), x1_3.reshape(-1, 1)), axis=1)

    # Create the target vector for class 1 (target is coded with one)
    X1_target = np.ones((m1,), dtype=np.int).reshape(-1, 1)


    #  Class 0 and 1 data are combined to create a single data matrix X
    X = np.append(X0, X1, axis=0)

    # Target values for class 0 & 1 are combined to create a single target vector
    y = np.concatenate((X0_target, X1_target), axis=0)
    
    return X, y.ravel()

In [7]:
# features array and target label array
X, y = data_generator()
print('X shape: %s and y.shape %s ' %(X.shape, y.shape))

X shape: (200, 3) and y.shape (200,) 


In [8]:
pd.DataFrame(X).sample(5)

,0,1,2
33,89.255183,62.528022,12.270969
190,46.260946,20.803942,22.896130
87,-31.340537,-33.848504,-13.061759
67,-69.299642,-57.668082,-42.714206
11,31.631274,23.228821,1.898840


####  Performance table with Minkowski distance

Below, a function create_table is written which calls best_performance function five different times to create the required table with best set of parameters as mentioned in the question, which are obtained by the cross-validation with GridSearchCV hyperparamteres tuning. Note that each call of best_performance function returns the performance metrics on a new set of randomly generated dataset. With the option scaling=True, it will implement the scaling of data, default case doesnot.

In [6]:
# retunrs a single row for table 1:
# does cross validation and returns the performance metrics with best parameters
def best_performance(scaling=False, seed=None):
    
    # data generation
    X, y = data_generator(seed)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)
    
    # if scaling is true
    if scaling:
        x_train = scale(x_train)
        x_test = scale(x_test)
    
    # GridSearchCV to find best hyperparameters
    n_neighbors = [1, 3, 5, 7, 17, 25, 35, 37]
    p = [1, 2, 4, 10, 20, 50, 100]
    weights = ["distance", "uniform"]
    param_grid = {'weights': weights, 'n_neighbors': n_neighbors, 'p': p}
    knn_cv = KNeighborsClassifier(metric='minkowski')
    knn_cv = GridSearchCV(knn_cv, param_grid=param_grid, scoring='f1', cv=5)
    knn_cv.fit(x_train, y_train)
    
    # Creating the k-NN model with best hyperparameters
    best_params = knn_cv.best_params_
    knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'], weights=best_params['weights'])
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    
    # perfromance metrics calculations
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # formatting the output: as a single row in the required table
    out = dict()    
    out['weights'] = best_params['weights']; out['n_neighbors'] = best_params['n_neighbors']; out['p'] = best_params['p']
    out['Accuracy'] =  accuracy; out['Precision'] = precision
    out['Recall'] = recall; out['F1-score'] = f1
    
    return out

# creates a table, a dataframe
# Note that five different seeds are used to get
# different set of data for each of the row in the table
def create_table(scaling=False):
    outs = dict()
    
    # each time with new seed it creates new dataset inside the function best_performance
    for seed in [13, 103, 1123, 1219, 137]:
        if scaling:
            ans = best_performance(scaling=True, seed=seed)
        else:
            ans = best_performance(scaling=False, seed=seed)
        for key, val in ans.items():
            if key not in outs.keys():
                outs[key] = [val]
            else:
                outs[key].append(val)
    # creating the final dataframe
    df = pd.DataFrame.from_dict(outs)
    # df = df.from_dict(outs)
    df.index = np.arange(1, 6)
    return df

In [7]:
create_table(scaling=False)

,weights,n_neighbors,p,Accuracy,Precision,Recall,F1-score
1,uniform,37,20,0.600,0.541667,0.722222,0.619048
2,distance,3,20,0.575,0.529412,0.500000,0.514286
3,uniform,25,4,0.500,0.454545,0.555556,0.500000
4,uniform,17,1,0.525,0.476190,0.555556,0.512821
5,distance,3,50,0.450,0.388889,0.388889,0.388889


#### Scaling Data

Note that the scaling is done in the same dataset used above in the absence of scaling, done by using fixed random seeds in the function create_table above.

In [8]:
create_table(scaling=True)

,weights,n_neighbors,p,Accuracy,Precision,Recall,F1-score
1,distance,37,10,0.450,0.423077,0.611111,0.500000
2,distance,3,10,0.675,0.631579,0.666667,0.648649
3,uniform,25,1,0.500,0.458333,0.611111,0.523810
4,uniform,17,1,0.525,0.476190,0.555556,0.512821
5,distance,3,2,0.525,0.478261,0.611111,0.536585


In this case, the scaling of the features does not warrent the improvements on the performance of the model. Since the dataset is produced randomly, it might not be always helpful to carry out the scaling. It depends upon the dataset, and we sometimes see the improvement after the scaling and while sometimes even decrement in the performance as seen in the above two tables.

####  Mahalanobis distance metric

Below we write a classify_with_mahalanobis function which takes a random seed as parameter and returns a single row in the required table. Each call of this function with an unique seed creates a different dataset. The search of the parameters is done by varying the number of nearest neighbors and weights parameters that the KNeighborsClassifier class takes. After brute force search of the best params, the performance measures on the test data set is returned.

In [10]:
# creates a single row in the required table
def classify_with_mahalanobis(seed):
    # new dataset
    X, y = data_generator(seed)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)
    
    # greedy searching for best k and weight
    minf1 = 0; params = dict()
    for k in np.arange(100, 162, 2):
        for weight in ['uniform', 'distance']:
            knn = KNeighborsClassifier(n_neighbors=k, weights=weight, algorithm='brute', 
                           metric='mahalanobis', metric_params={'V': np.cov(x_train)})
            knn.fit(x_train, y_train)
            y_pred = knn.predict(x_test)
            f1 = accuracy_score(y_test, y_pred)
            if f1 > minf1:
                params['f1_score'] = f1; params['k'] = k; params['weights'] = weight
                minf1 = f1
    
    # now let us find metrics with the best params model
    knn = KNeighborsClassifier(n_neighbors=params['k'], weights=params['weights'], algorithm='brute', 
                           metric='mahalanobis', metric_params={'V': np.cov(x_train)})
    knn.fit(x_train, y_train)
    # prediction
    y_pred = knn.predict(x_test)
        
    # perfromance metrics calculations
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # formatting the output: goes as a single row in the required table
    out = dict()    
    out['weights'] = weight; out['n_neighbors'] = k
    out['Accuracy'] =  accuracy; out['Precision'] = precision
    out['Recall'] = recall; out['F1-score'] = f1
        
    return out


# creates a table with classifier_with_mahalanobis
# creates new dataset for each of the rows
# the dataset will be fixed for the fixed seed values used below.
def create_table():
    outs = dict()
    for seed in [202, 44, 10120, 212, 100]:
        ans = classify_with_mahalanobis(seed=seed)
        for key, val in ans.items():
            if key not in outs.keys():
                outs[key] = [val]
            else:
                outs[key].append(val)
    # creating the final dataframe
    df = pd.DataFrame.from_dict(outs)
    # df = df.from_dict(outs)
    df.index = np.arange(1, 6)
    return df

In [11]:
# required performance table for part 3.
create_table()

,weights,n_neighbors,Accuracy,Precision,Recall,F1-score
1,distance,160,0.750,0.681818,0.833333,0.750000
2,distance,160,0.650,0.590909,0.722222,0.650000
3,distance,160,0.575,0.517241,0.833333,0.638298
4,distance,160,0.675,0.600000,0.833333,0.697674
5,distance,160,0.600,0.538462,0.777778,0.636364


#### Comparision of Minkowski and Mahalanobis Distances

From the many experiments done in both the minkowski and mahalanobis distances, we find that mahalanobis distance does better performance. This has been seen from the cross-validation hyperparameters tuning using minkowski distance, using GridSearchCV, and mahalanobis distance using the custom greedy parameters search.

As expected, in a randomly created/distributed dataset, mahalanobis distance usually does better. mahalanobis distance is also used to identify the outlier in dataset. However, the problem with Mahalanobis distance is that, it becomes computationally very expensive for larger and high dimensional dataset as it needs to compute the inverse of the covaraince matrix. 

# Theoretical Analysis of Data Covariance

#### Importing data: 

Here, we use the winequality-white.csv dataset - the same data used in previous notebooks.
From the complete dataset as a DataFrame, let us create a feature maxtix X and target array y. 

In [12]:
df = pd.read_csv('winequality-white.csv', sep=';')

y = df['quality']
X = df.drop(columns=['quality'], inplace=False).values

In [13]:
X.shape # n*d

(4898, 11)

In [14]:
covX = np.cov(X)
print('Dimension of covarance matrix of X: ', covX.shape)

Dimension of covarance matrix of X:  (4898, 4898)


The inverse of covaraince matrix of X:

In [17]:
linalg.inv(covX)

LinAlgError: Singular matrix

Here, we are getting the covX as a singular matrix because the numpy function incorrectly calculates its determinant 0. This is not correct as we check from the tests done for a real singular matrix, done **below**  with the $2\times2$ matrix given in the updated recitation note.

In [20]:
covxx = pd.DataFrame(covX)

In [21]:
v, _ = linalg.eig(covxx.values)

In [22]:
for val in v:
    if abs(val) == 0.0:
        print(val)

Here, it turns out that non of the eigenvalues of the covaraince matrix is 0.

In [23]:
linalg.det(covX)

0.0

In [24]:
eigvalues, eigvectors = linalg.eig(covxx)

In [25]:
# checking if any of the eigenvectors are zeros or not
for i in range(len(eigvectors)):
    for j in range(len(eigvectors[i])):
        if eigvectors[i][j] == 0.:
            print(0.)

**No eigenvectors is zero: so it might not be singular, as we are getting the det 0 because of some rounding error.**

#### Positive-definite or semi-definite?

In [26]:
A = [[1, 2], [2, 4]]

# a random vector of length len(A)
x = np.random.randint(0, 10, (len(A)))
xT = x.reshape(len(A), 1)

xTAx = np.matmul(np.matmul(x, A), xT)
print('xTAx: ', xTAx)

print('===========')
print('checking: ')
xTAx >= 0

xTAx:  [400]
checking: 


array([ True])

In [27]:
# with above covariance matrix
A = covX

In [28]:
linalg.det(A)

0.0

In [29]:
# checking the smaller sub-matrices of the large A
for i in range(1, 30):
    Di = np.array([list(row[:i]) for row in A[:i]])
    if i == 1:
        D1 = Di[0][0]
        print('det[D1]: ', D1)
    detDi = linalg.det(Di)
    print('det[D%d]: %f' % (i, detDi), sep=', ', flush=True, end=', ')

det[D1]:  2550.334776454546
det[D1]: 2550.334776, det[D2]: 143612.728148, det[D3]: 1080266.369765, det[D4]: 5064071.143343, det[D5]: 0.000000, det[D6]: 0.000000, det[D7]: 0.000000, det[D8]: 0.000000, det[D9]: 0.000000, det[D10]: 0.000000, det[D11]: 0.000000, det[D12]: 0.000000, det[D13]: 0.000000, det[D14]: 0.000000, det[D15]: 0.000000, det[D16]: 0.000000, det[D17]: 0.000000, det[D18]: 0.000000, det[D19]: 0.000000, det[D20]: 0.000000, det[D21]: 0.000000, det[D22]: 0.000000, det[D23]: 0.000000, det[D24]: 0.000000, det[D25]: 0.000000, det[D26]: -0.000000, det[D27]: 0.000000, det[D28]: 0.000000, det[D29]: -0.000000, 

Notice that the determinants of all the sub-matrices that are larger than size 4 are 0.
This is surprising: let us explore what's wrong with it. It seems there is something wrong in the 
output of the calculation coming from the python itself.

In [30]:
A4 = np.array([list(row[:4]) for row in A[:4]])
linalg.det(A4)

5064071.143342724

In [31]:
A5 = np.array([list(row[:5]) for row in A[:5]])
linalg.det(A5)

0.0

What about the cov(X) matrix?

In [32]:
# a random vector of length len(A)
x = np.random.randint(0, 10, (len(A)))
xT = x.reshape(4898, 1)

xTAx = np.matmul(np.matmul(x, A), xT)
print('xTAx: ', xTAx)

print('===========')
print('checking: ')
xTAx >= 0

xTAx:  [8.29280492e+11]
checking: 


array([ True])

***Yes,  A (Cov(X)) is positive-semidefinite.***

####  Removing singularity of data matrix:

Introduction of small noise into the dataset can remove the singlularity of the matrix slightly changing the determinant of the matrix.

But the better way of solving this problem is to decrease the dimnesion of the dataset, i.e. to get rid of some of the redundant features in the dataset so that it becomes non-singular. It can be done using (Principle Component Analysis) PCA. 

####  Time compexity of inverting a matrix:

The airthmetic time compelxity of inversion of a matrix of size ($n \times n$) using Gauss-Jordan elimination method is **O**(n$^3$). There are slightly better algorithms for the matrix inversion.

#### Checking out with a simple singular $2 \times 2$ matrix

In [33]:
A = [[1, 2], [2, 4]]
np.array(A)

array([[1, 2],
       [2, 4]])

In [34]:
# eigenvalues and eigenvectors

eigvalues, eigvectors = linalg.eig(A)

In [35]:
eigvalues

array([0., 5.])

In [36]:
eigvectors

array([[-0.89442719, -0.4472136 ],
       [ 0.4472136 , -0.89442719]])

In [37]:
linalg.det(A)

0.0

Here, out of the two eigenvalues, one of them is zero. One thing we can learn from this is that a real singular matrix will have at least one of its eigenvalue zero.